# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col='City_ID')

# Display sample data
city_data_df.head()

,City,Latitude,Longitude,Max Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/sec),Country,Date
City_ID,,,,,,,,,
0,college,64.8569,-147.8028,-4.50,84,75,0.00,US,1698527978
1,bilibino,68.0546,166.4372,-22.35,97,32,0.95,RU,1698527978
2,pelabuhanratu,-6.9875,106.5514,24.57,87,37,1.86,ID,1698527978
3,atafu village,-8.5421,-172.5159,28.38,75,30,4.05,TK,1698527979
4,grytviken,-54.2811,-36.5092,-0.02,94,100,1.65,GS,1698527979


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Longitude',
    y='Latitude',
    geo=True,
    size='Humidity (%)',
    tiles='OSM',
    c='City',
    cmap='tab20',
    height=800,
    width=1200,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit defined criteria and drop any results with null values
condition_temp_high = city_data_df['Max Temperature (C)'] <= 28
condition_temp_low = city_data_df['Max Temperature (C)'] >= 24
condition_hum_high = city_data_df['Humidity (%)'] <= 70
condition_hum_low = city_data_df['Humidity (%)'] >= 30
condition_country_bad = ['AF', 'BY', 'BF','CF', 'MM', 'PS', 'HT', 'IR', 'IQ', 'LB', 'LY', 'ML', 'MX', 'NE', 'KP', 'PK','RU', 'SO', 'SS', 'SD', 'SY', 'UA', 'VE', 'YE']

best_cities_df = city_data_df[condition_temp_high & condition_temp_low & condition_hum_high & condition_hum_low & (~city_data_df['Country'].isin(condition_country_bad))]

# Drop any rows with null values
best_cities_df_cleaned = best_cities_df.dropna()

# Display sample data
best_cities_df_cleaned.head()

,City,Latitude,Longitude,Max Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/sec),Country,Date
City_ID,,,,,,,,,
10,bethel,41.3712,-73.4140,26.05,63,0,2.68,US,1698527980
23,ocean city,39.2776,-74.5746,27.14,61,0,3.65,US,1698527982
30,stratford,41.1845,-73.1332,27.12,63,20,2.24,US,1698527823
73,pimentel,-6.8367,-79.9342,24.16,64,20,9.26,PE,1698527988
75,wailua homesteads,22.0669,-159.3780,27.85,66,0,3.60,US,1698527988


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_df_cleaned[['City', 'Country', 'Latitude', 'Longitude', 'Humidity (%)']].reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,10,bethel,US,41.3712,-73.4140,63,
1,23,ocean city,US,39.2776,-74.5746,61,
2,30,stratford,US,41.1845,-73.1332,63,
3,73,pimentel,PE,-6.8367,-79.9342,64,
4,75,wailua homesteads,US,22.0669,-159.3780,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    coordinates_lat = hotel_df.loc[index, "Latitude"]
    coordinates_long = hotel_df.loc[index, "Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{coordinates_long},{coordinates_lat},{radius}"
    params["bias"] = f"proximity:{coordinates_long},{coordinates_lat}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
ocean city - nearest hotel: Watson's Regency Suites
stratford - nearest hotel: Residence Inn Milford
pimentel - nearest hotel: Hospedaje Calle del Sol
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
holualoa - nearest hotel: Kona Hotel
suwayr - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
gewane - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
maghar - nearest hotel: Mantur by Selina
karratha - nearest hotel: Karratha International Hotel
kill devil hills - nearest hotel: Mariner Days Inn & Suites
sur - nearest hotel: Sur Hotel
palmer - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
sechura - nearest hotel: Hospedaje Claritza
carroll - nearest hotel: Days Inn
iaciara - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
cockburn town 

,City_ID,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,10,bethel,US,41.3712,-73.4140,63,Hampton Inn Danbury
1,23,ocean city,US,39.2776,-74.5746,61,Watson's Regency Suites
2,30,stratford,US,41.1845,-73.1332,63,Residence Inn Milford
3,73,pimentel,PE,-6.8367,-79.9342,64,Hospedaje Calle del Sol
4,75,wailua homesteads,US,22.0669,-159.3780,66,Hilton Garden Inn Kauai Wailua Bay
5,79,holualoa,US,19.6228,-155.9522,68,Kona Hotel
6,92,suwayr,SA,30.1171,40.3893,33,No hotel found
7,101,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
8,113,gewane,ET,10.1664,40.6453,68,No hotel found
9,120,west island,CC,-12.1568,96.8225,69,Cocos Village Bungalows


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Longitude',
    y='Latitude',
    geo=True,
    size='Humidity (%)',
    tiles='OSM',
    c='City',
    cmap='tab20',
    height=800,
    width=1200,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%),Hotel Name,Country)